In [4]:
from langame.langame_client import LangameClient
from langame.array import intersection
from pprint import pprint
from firebase_admin import firestore
import requests
from bs4 import BeautifulSoup
import json
import openai
import random

In [5]:
c = LangameClient(path_to_config_file="../config.yaml")
memes = []
for e in c._firestore_client.collection('memes').stream():
    memes.append((e.id, e.to_dict()))


In [7]:
import time
from tqdm import tqdm
from random import randint
def get_prompt(topics):
    rnd = random.choices(memes, k=500)
    return [{"content": e[1]['content'], "topics": e[1]['topics']} for e in rnd if len(intersection(e[1]['topics'], topics)) > 0]
outputs = []
for i in tqdm(range(10**12)):
    time.sleep(randint(1,3))
    samples = get_prompt(['ice breaker'])
    p = "\n".join([json.dumps(e) for e in samples[0:60]])
    try:
        response = openai.Completion.create(
            engine="davinci-codex",
            prompt=p+"\n",
            temperature=1,
            max_tokens=100,
            top_p=1,
            frequency_penalty=0.7,
            presence_penalty=0,
            stop=["\n"]
        )
    except Exception as e:
        print(e)
        continue
    if response["choices"][0]["finish_reason"] == "length":
        continue
    text = response["choices"][0]["text"]
    outputs.append(text)
    # every 10 samples, append the generated samples to file
    if i % 10 == 0:
        with open("generated_samples.txt", "a") as f:
            f.write("\n".join(outputs) + "\n")
            outputs = []

  0%|          | 1/1000000000000 [00:04<1276401612:29:32,  4.60s/it]

{"content": "What is one thing you would never think of telling someone else?", "topics": ["mind", "ice breaker"]}


  0%|          | 2/1000000000000 [00:09<1289429473:43:06,  4.64s/it]

{"content": "What is the joke that you've heard the most times and always find yourself laughing?", "topics": ["other"]}


  0%|          | 3/1000000000000 [00:14<1335375518:33:19,  4.81s/it]

{"content": "What two or three things should a company hire people in advance to guarantee they're going to be successful?", "topics": ["ice breaker"]}


  0%|          | 4/1000000000000 [00:19<1396712947:39:21,  5.03s/it]

{"content": "What \u201cun-cool\u201d thing are you most guilty of doing as a kid?", "topics": ["ice breaker"]}


  0%|          | 4/1000000000000 [00:20<1437050534:36:48,  5.17s/it]


KeyboardInterrupt: 